In [21]:
library(RPostgreSQL)
library(dplyr)
library(GenomicRanges)
library(doParallel)
library(dbplyr)
library(BiocParallel)

In [2]:
source("../BDDS/footprints/testdb/src/dbFunctions.R")

In [3]:
# connect to databases

# load chipseq traditionally -- will just put this in memory eventually
if(!exists("db.chipseq"))
    dbname="chipseq"
    port="5432"
    driver=dbDriver("PostgreSQL")
    user="trena"
    password="trena"
    host="whovian"
    db.chipseq <- dbConnect(drv=driver, user=user, password=password, dbname=dbname, host=host, port=port)

Here, we've connected to our database of ChipSeq data; it's our "gold standard" for comparison and it doesn't contain that many different different TFs.

## create map: M[TF] = c(motif1,motif2,...)

In [4]:
# chipseq hits table can fit in memory so load it up
chipseq.hits <- dbGetQuery(db.chipseq, "select * from hits")
chipseq.hits <- as.tbl(chipseq.hits)

# get chipseq regions data and change chrom notations from eg chr10 to just 10 as in fimo
chipseq.regions <- dbGetQuery(db.chipseq, "select * from regions")
chipseq.regions <- as.tbl(chipseq.regions)
chr.list <- chipseq.regions$chrom
cutoff <- nchar("chr")+1
no.chr.list <- substring(chr.list,cutoff)
chipseq.regions$chrom <- no.chr.list

In [5]:
# see which of the TFs in the chipseq data are mapped to any motifs
#TF.motif.pairs <- read.csv("text_data_files/motif_to_tf_mappings_with_tfclass_include_multiple.csv",
#                            stringsAsFactors=FALSE)

# Change the file to the new mapping and change column names to wha they were before
    TF.motif.pairs <- readRDS("../BDDS/trenadb/fimo/2017_08_23_Motif_TF_Map.RDS")
names(TF.motif.pairs) <- c("motif","tfs")

unique.cs.tfs <- unique(chipseq.hits$name)
cs.tf.nomatches <- unique.cs.tfs[!(unique(unique.cs.tfs) %in% unique(TF.motif.pairs$tfs))]
cs.tf.matches <- unique.cs.tfs[unique(unique.cs.tfs) %in% unique(TF.motif.pairs$tfs)]
head(TF.motif.pairs)

motif,tfs
Mmusculus-UniPROBE-Sp100.UP00049,SP140L
Mmusculus-jaspar2016-Hes1-MA1099.1,HEY2
Mmusculus-jaspar2016-Hes1-MA1099.1,HEY1
Mmusculus-jaspar2016-Hes1-MA1099.1,HES7
Mmusculus-jaspar2016-Hes1-MA1099.1,HES6
Mmusculus-jaspar2016-Hes1-MA1099.1,HES5


In [6]:
length(unique.cs.tfs)
length(cs.tf.nomatches)
length(cs.tf.matches)

[1] 77

[1] 5

[1] 72

**NOTE: Previously we got 60; now we get 72**

Up to this point we've done this:

1. Read all our hits from ChipSeq 
2. Read in all our Motif-TF mappings
3. Pulled out the unique TFs from ChipSeq (there are 77) and found those that are in our Motif-TF mapping (there are 60)

Now we want to use our motif-TF mapping to make a list of all TFs and their motifs AND to make a list of all motifs and their TFs. This will necessarily be 2 lists of lists; as Rory notes, a dictionary would be better, but this is R, not Python

In [7]:
# create a list of lists (poor sub for dict) for TFs -> motifs and motifs -> TFs
# only use jaspar motifs

#TF.motif.pairs <- as.tbl(read.csv("text_data_files/motif_to_tf_mappings_with_tfclass_include_multiple.csv",
#                            stringsAsFactors=FALSE))

TF.motif.pairs <- as.tbl(TF.motif.pairs)
TF.motif.pairs.jaspar <- subset(TF.motif.pairs, grepl("MA\\d{4}\\.\\d$", motif) )

TFs.to.motifs <- list()
for (TF in cs.tf.matches) {
    this.TF.df <- subset(TF.motif.pairs.jaspar, tfs %in% TF)
    TFs.to.motifs[TF] <- list(this.TF.df$motif)
}
TFs.to.motifs <- TFs.to.motifs[lapply(TFs.to.motifs,length) >= 1]

motifs.to.TFs <- list()
for (mtf in unique(TF.motif.pairs.jaspar$motif)) {
    this.motif.df <- subset(TF.motif.pairs.jaspar, motif %in% mtf)
    motifs.to.TFs[mtf] <- list(this.motif.df$tfs)
}
motifs.to.TFs <- motifs.to.TFs[lapply(motifs.to.TFs,length) >= 1]

In [8]:
nrow(TF.motif.pairs.jaspar)

[1] 11416

In [9]:
length(TFs.to.motifs)
length(motifs.to.TFs)

[1] 62

[1] 631

What this tells us is that when we look at only JASPAR motifs:

* There are only 55 TFs in our ChipSeq set that are JASPAR motifs
* There are 514 unique JASPAR motifs in our TF-motif mapping

We'll save these to a couple maps that we can load/use later. We also don't actually know how many motifs are mapped to by our TFs; we only know the total number of JASPAR motifs and the total number of TFs that have JASPAR motifs. 

We'll quickly run through our TF-to-motif mapping and see how many unique motifs are mapped to by our TFs

In [10]:
save(TFs.to.motifs, motifs.to.TFs, file="Rdata_files/Tfmotifmap.Rdata")
# load("Rdata_files/Tfmotifmap.Rdata")

In [11]:
allmots <- c()

for (TFname in names(TFs.to.motifs)) {
    allmots  <-  c(allmots, TFs.to.motifs[[TFname]])
}
length(unique(allmots))

[1] 264

In [12]:
# write.table(unique(allmots), file="text_data_files/unique_motifs_for_CS_TFs.txt", row.names=FALSE, col.names=FALSE, quote=FALSE)

So as we can summarize our "gold-standard" ChipSeq data now as follows:

* Our ChipSeq data contain 77 TFs, 60 of which we have a mapping for and 55 of which are JASPAR
* Mapping our 55, we can get 195 unique motifs

## function to output df containing pos and neg examples for each TF

Here's what we actually do for a given TF

1. We pull out all ChipSeq hits for the TF and their associated regions
2. Using our mapping, we look for ALL the occurences of the motifs for that TF in the FIMO database
3. We find the overlaps of (1) and (2), so now we have the regions in FIMO where there's overlap with ChipSeq hits. And this is for a given TF
4. Using our overlaps, we grab the FIMO motifs that are part of the overlaps; these are our "positives"
5. Using a ratio of neg/pos, we find the "positives" for each motif of the TF and multiply it by the ratio. So if we have neg/pos = 0.5 and there are 10 hits (positives) of a particular motif, we want 5 negatives. 
6. We then find the actual "negatives" for each motif by finding those that don't pop up in the positives. 
7. Using our "wanted" number of negatives and our "actual", we take the smaller of these 2 (so we get our wanted number, unless there's fewer than that) and then sample the negatives to grab this many for each motif. 
8. Finally, we put the positive and negative together, with a column called "cs_hit" designating a positive or negative

More succinctly, for each TF we:

1. Find all the motifs for a TF from our mapping.
2. Pull out all the places where these motifs in FIMO overlap the ChipSeq data
3. Compile a dataset of places where we had hits and then a sampling of the same motifs in FIMO where we didn't overlap ChipSeq data


In [13]:
create.TF.df <- function(TF, neg.pos.ratio=10, verbose=FALSE) {
    
    # Note: this is how you make a DB connection in dplyr
    db.fimo.dplyr <- src_postgres(drv=dbDriver("PostgreSQL"),
                                  user="trena",
                                  password="trena",
                                  dbname="fimo",
                                  #host="whovian", #Changing this to khaleesi
                                  #port="5432")
                                  host = "localhost")
    tbl.fimo.dplyr <- tbl(db.fimo.dplyr, "fimo_hg38")
    
    # regions locs we can compute on but hits have TF info so need both
    # Here, we grab all hits for a TF, then grab the regions of those hits
    chipseq.hits.TF <- subset(chipseq.hits, name == TF)
    locs.TF <- chipseq.hits.TF$loc
    chipseq.regions.TF <- subset(chipseq.regions, loc %in% locs.TF)
    
    # next step is slow, gives you some context
    if (verbose == TRUE) {
        if (length(TFs.to.motifs[[TF]])==1) {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motif"))
        } else {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motifs"))
        }       
    }
        
    # this is the slow step -- doing SQL queries on tbl.fimo.dplyr = call to whole fimo database    
    # need branch since %in% conversion to SQL doesn't work on length == 1
    ## Basically: we find all instances of the TF's motif(s) in FIMO
    if (length(TFs.to.motifs[[TF]]) > 1 ) {
        fimo.motifs.for.TF <- as.tbl(as.data.frame(filter(tbl.fimo.dplyr, motifname %in% TFs.to.motifs[[TF]])))
    } else {
        fimo.motifs.for.TF <- as.tbl(as.data.frame(filter(tbl.fimo.dplyr, motifname  ==  TFs.to.motifs[[TF]])))
    }
    
    # find intersect using fast genomic ranges data structure
    # We make GR objects for the fimo motifs we just found and for the chipseq regions, then find their overlaps
    gr.fimo.TF <- with(fimo.motifs.for.TF, GRanges(chrom, IRanges(start=start, end=endpos)))
    gr.chipseq.TF <- with(chipseq.regions.TF, GRanges(chrom, IRanges(start=start, end=endpos)))
    overlaps.gr.TF <- findOverlaps(gr.chipseq.TF, gr.fimo.TF, type="any")
    overlaps.TF <- as.tbl(as.data.frame(overlaps.gr.TF))
    
    # row numbers in fimo.motifs.for.TF where motifs overlap with chipseq peaks
    positive.fimo.examples.rows.TF <- unique(overlaps.TF$subjectHits)
    positive.fimo.examples.TF.df <- fimo.motifs.for.TF[positive.fimo.examples.rows.TF,]
    
    # figure out how many negative samples for each motif we want
    tot.motif.counts.TF <- table(fimo.motifs.for.TF$motifname)
    pos.motif.counts.TF <- table(fimo.motifs.for.TF[positive.fimo.examples.rows.TF,]$motifname)
    nx.pos.motif.counts.TF <- pos.motif.counts.TF*neg.pos.ratio

    # want neg samples in fimo.motifs.for.TF to be non overlapping with chipseq peaks or pos examples
    neg.cands.for.single.TF.df <- subset(fimo.motifs.for.TF,
                                         !(start %in% positive.fimo.examples.TF.df$start) &
                                         !(endpos %in% positive.fimo.examples.TF.df$endpos))
    neg.motif.counts.TF <- table(neg.cands.for.single.TF.df$motifname)

    # don't try to sample more than the population
    neg.sample.counts.TF <- pmin(nx.pos.motif.counts.TF,neg.motif.counts.TF)
    
    # for each motif this TF matches, sample some examples where no CS hit
    negative.fimo.examples.TF.df <- tibble()
    for (motname in names(neg.sample.counts.TF)) {
        neg.cands.for.single.TF.df.single.motif <- subset(neg.cands.for.single.TF.df, motifname == motname)
        negative.fimo.examples.TF.df <- rbind(negative.fimo.examples.TF.df,
                                    sample_n(neg.cands.for.single.TF.df.single.motif, neg.sample.counts.TF[[motname]]))
    }
    
    # annotate and collect all samples
    positive.fimo.examples.TF.df <- as.tbl(cbind(positive.fimo.examples.TF.df, "cs_hit"=1))
    negative.fimo.examples.TF.df <- as.tbl(cbind(negative.fimo.examples.TF.df, "cs_hit"=0))
    all.fimo.examples.TF.df <- as.tbl(rbind(positive.fimo.examples.TF.df,negative.fimo.examples.TF.df))
    
    return(all.fimo.examples.TF.df)
    
}

## create df of pos/neg samples for all TFs all together

With our function defined, we'll now pull out the positive and negative samples for each TF; we're doing so with a neg/pos ration of 9. We're also doing this using a parallel strategy. The output gets saved in an `RData` file

In [14]:
# ratio of negative to positve examples in data set
pnr=9

In [15]:
# parallel loop over all TFs
sorted.TF.names <- sort(names(TFs.to.motifs))
N.TF <- length(sorted.TF.names)

register(MulticoreParam(workers = 20, stop.on.error = FALSE, log = TRUE), default = TRUE)
all.TF.df <- bplapply(sorted.TF.names, create.TF.df, neg.pos.ratio = pnr, verbose = TRUE)
all.TF.df <- bind_rows(all.TF.df)

# save data
fname=paste0("/ssd/mrichard/data/all.TF.fimo.samples.ratio.",pnr,".df.RData")
save(all.TF.df, file=fname)

############### LOG OUTPUT ###############
Task: 20
Node: 20
Timestamp: 2017-09-01 14:37:57
Success: TRUE
Task duration:
    user   system  elapsed 
  11.228    2.548 1260.046 
Memory used:
           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  2380203 127.2    3886542 207.6  3886542 207.6
Vcells 35442038 270.5   89160810 680.3 89158815 680.3
Log messages:
stderr and stdout:
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
############### LOG OUTPUT ###############
Task: 1
Node: 1
Timestamp: 2017-09-01 14:38:33
Success: TRUE
Task duration:
    user   system  elapsed 
  41.520    7.356 1294.740 
Memory used:
           used  (Mb) gc trigger  (Mb) max used  (Mb)
Ncells  3193454 170.6    4703850 251.3  3886542 207.6
Vcells 46087320 351.7   89160

############### LOG OUTPUT ###############
Task: 12
Node: 12
Timestamp: 2017-09-01 14:44:30
Success: TRUE
Task duration:
    user   system  elapsed 
 157.972   45.484 1650.379 
Memory used:
            used   (Mb) gc trigger   (Mb)  max used   (Mb)
Ncells  14140659  755.2   20885653 1115.5  14157883  756.2
Vcells 195094662 1488.5  320512570 2445.4 195100871 1488.6
Log messages:
stderr and stdout:
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
R_zmq_msg_send errno: 4 strerror: Interrupted system call
############### LOG OUTPUT ###############
Task: 6
Node: 6
Timestamp: 2017-09-01 14:46:11
Success: TRUE
Task duration:
    user   system  elapsed 
 216.168   76.568 1751.765 
Memory used:
            used   (Mb) gc trigger   (Mb)  max used   (Mb)
Ncells  14745982  787.6   25102783 1340.7  14763207  788.5
Vcel

ERROR: Error in eval_bare(dot$expr, dot$env): object 'all.Tf.df' not found


In [18]:
all.TF.df <- bind_rows(all.TF.df)

In [19]:
head(all.TF.df)
fname=paste0("/ssd/mrichard/data/all.TF.fimo.samples.ratio.",pnr,".df.RData")
save(all.TF.df, file=fname)

motifname,chrom,start,endpos,strand,motifscore,pval,empty,sequence,loc,cs_hit
Mmusculus-jaspar2016-Nfe2l2-MA0150.2,1,1677938,1677952,+,13.34830,1.13e-05,,CACTGTGACTCCGCA,chr1:1677938-1677952,1
Mmusculus-jaspar2016-Bach1::Mafk-MA0591.1,1,1677939,1677953,+,11.70790,1.37e-05,,ACTGTGACTCCGCAG,chr1:1677939-1677953,1
Hsapiens-jaspar2016-JUND(var.2)-MA0492.1,1,1828558,1828572,-,9.73034,9.18e-05,,TTTTGTGAGGTCACT,chr1:1828558-1828572,1
Hsapiens-jaspar2016-ATF7-MA0834.1,1,2255916,2255929,-,8.03371,6.68e-05,,AAATGAGGTCACTG,chr1:2255916-2255929,1
Hsapiens-jaspar2016-JUND(var.2)-MA0492.1,1,2255789,2255803,-,13.71910,1.48e-05,,GGAAATGAGGTCAGG,chr1:2255789-2255803,1
Hsapiens-jaspar2016-JUND(var.2)-MA0492.1,1,2255823,2255837,-,13.71910,1.48e-05,,GGAAATGAGGTCAGG,chr1:2255823-2255837,1


In [20]:
str(all.TF.df)

Classes ‘tbl_df’, ‘tbl’ and 'data.frame':	24720680 obs. of  11 variables:
 $ motifname : chr  "Mmusculus-jaspar2016-Nfe2l2-MA0150.2" "Mmusculus-jaspar2016-Bach1::Mafk-MA0591.1" "Hsapiens-jaspar2016-JUND(var.2)-MA0492.1" "Hsapiens-jaspar2016-ATF7-MA0834.1" ...
 $ chrom     : chr  "1" "1" "1" "1" ...
 $ start     : int  1677938 1677939 1828558 2255916 2255789 2255823 2255891 2255917 2255949 2255983 ...
 $ endpos    : int  1677952 1677953 1828572 2255929 2255803 2255837 2255905 2255931 2255963 2255997 ...
 $ strand    : chr  "+" "+" "-" "-" ...
 $ motifscore: num  13.35 11.71 9.73 8.03 13.72 ...
 $ pval      : num  1.13e-05 1.37e-05 9.18e-05 6.68e-05 1.48e-05 1.48e-05 1.48e-05 8.28e-06 1.48e-05 1.48e-05 ...
 $ empty     : chr  " " " " " " " " ...
 $ sequence  : chr  "CACTGTGACTCCGCA" "ACTGTGACTCCGCAG" "TTTTGTGAGGTCACT" "AAATGAGGTCACTG" ...
 $ loc       : chr  "chr1:1677938-1677952" "chr1:1677939-1677953" "chr1:1828558-1828572" "chr1:2255916-2255929" ...
 $ cs_hit    : num  1 1 1 1 1 1 1 1

## serial version of production loop

This is the same thing, but done serially instead of with parallel computing. Not much to see here. 

**Don't run this if you've already run the one above; it takes FOREVER!!!**

In [ ]:
# serial version -- hung on last TF, don't know why

all.TF.df <- tibble()

sorted.TF.names <- sort(names(TFs.to.motifs))
for (TFname in sorted.TF.names) {
    
    TFnum <- which(sorted.TF.names %in% TFname)
    message(paste("Processing TF", TFnum,"/", length(sorted.TF.names)))
    TF.df <- create.TF.df(TFname, neg.pos.ratio=9, verbose=TRUE)
    all.TF.df <- rbind(all.TF.df,TF.df)
    
    save(all.TF.df, file="/ssd/mrichard/data/all.TF.fimo.samples.ratio.9.df.RData")

}

# TESTING AREA

In [33]:
# Grab tsorted.TF.nameswhich has 1 motif, and use it
sorted.TF.names <- sort(names(TFs.to.motifs))
TF <- sorted.TF.names[62] 
system.time(TF.df <- create.TF.df(TF, neg.pos.ratio=9, verbose=TRUE))

Auto-disconnecting PostgreSQLConnection
ZNF143 - querying fimo database for 1 motif


   user  system elapsed 
  2.460   0.284 147.267 

In [30]:
verbose <- TRUE 
neg.pos.ratio <- 9

db.fimo.dplyr <- src_postgres(drv=dbDriver("PostgreSQL"),
                                  user="trena",
                                  password="trena",
                                  dbname="fimo",
                                  #host="whovian", #Changing this to khaleesi
                                  #port="5432")
                                  port = "5432",
                                  host = "localhost")
    tbl.fimo.dplyr <- tbl(db.fimo.dplyr, "fimo_hg38")
    
    # regions locs we can compute on but hits have TF info so need both
    # Here, we grab all hits for a TF, then grab the regions of those hits
    chipseq.hits.TF <- subset(chipseq.hits, name == TF)
    locs.TF <- chipseq.hits.TF$loc
    chipseq.regions.TF <- subset(chipseq.regions, loc %in% locs.TF)
    
    # next step is slow, gives you some context
    if (verbose == TRUE) {
        if (length(TFs.to.motifs[[TF]])==1) {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motif"))
        } else {
            message(paste(TF, "- querying fimo database for", length(TFs.to.motifs[[TF]]), "motifs"))
        }       
    }

ZNF274 - querying fimo database for 0 motifs
